## 1. Baseline MLP on Cora (Ignoring Graph Structure)

- Use only the **node features `X`** to predict the node labels.
- **Ignore the graph structure** entirely (we do not use the adjacency matrix `A`).

This baseline will give us a reference performance to compare against more advanced **Graph Neural Network (GNN)** models later.

### Why an MLP baseline?

- It treats each node **independently**, as if we shuffled and removed all edges.
- It can only learn from each node's own feature vector, **not from its neighbors**.
- Performance is usually **worse than GNNs**, because citation relationships (graph structure) are very informative for classification.

## 2. Load Preprocessed Cora Data from Disk

We previously saved the Cora dataset to the `../data/` directory in another notebook. Here, we will:

- Load the saved NumPy arrays for:
  - Node features `X.npy`
  - Labels `y.npy`
  - Adjacency matrix `A.npz` (loaded for completeness, but **not used** by the MLP)
  - Train / validation / test masks
- Confirm their shapes.

> **Important:** We do **not** call Spektral's dataset loader here. All data comes from the saved files.
